In [3]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 準備圖片集

In [9]:
# 讀取檔案位置跟label`
train_path = pd.read_csv('image_list_with_label.csv')
train_path.head()

,feature,label
0,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,bedroom
1,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,bedroom
2,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,bedroom
3,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,bedroom
4,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,bedroom


In [10]:
#Shuffle the label
train_path = train_path.sample(frac=1)
train_path.head()

,feature,label
987,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,industrial
2935,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,tallbuilding
1655,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,mountain
2788,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,tallbuilding
833,C:\Users\AdamChang\Documents\Python\Data\AIA_M...,highway


In [11]:
import os
from skimage import io, transform

In [12]:
label = train_path['label']

In [13]:
target = set(label); target

{'CALsuburb',
 'PARoffice',
 'bedroom',
 'coast',
 'forest',
 'highway',
 'industrial',
 'insidecity',
 'kitchen',
 'livingroom',
 'mountain',
 'opencountry',
 'store',
 'street',
 'tallbuilding'}

In [14]:
#建立classes的index
classes = {name:(idx+1) for idx,name in enumerate(target)}

In [15]:
classes

{'CALsuburb': 11,
 'PARoffice': 13,
 'bedroom': 4,
 'coast': 6,
 'forest': 5,
 'highway': 12,
 'industrial': 8,
 'insidecity': 7,
 'kitchen': 1,
 'livingroom': 15,
 'mountain': 14,
 'opencountry': 2,
 'store': 10,
 'street': 3,
 'tallbuilding': 9}

In [16]:
#將label 的文字轉成數字的class
labels_map = label.map(classes)

In [17]:
labels_map.head()

987      8
2935     9
1655    14
2788     9
833     12
Name: label, dtype: int64

In [18]:
train_labels = np.array(labels_map)

In [19]:
train_labels[0:5]

array([ 8,  9, 14,  9, 12], dtype=int64)

In [21]:
#讀取圖片並建立成lst
img_lst = np.empty(shape=[train_path['feature'].shape[0],224,224,3])
for i, path in enumerate(train_path['feature']):
    img = io.imread(path)
    img_resize = transform.resize(img, output_shape=(224,224,3))
    img_lst[i] = img_resize

C:\Users\AdamChang\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [62]:
x_train, y_train = img_lst[0:2], train_labels[0:2]
x_valid, y_valid = img_lst[2:3], train_labels[2:3]
x_test, y_test = img_lst[3:4], train_labels[3:4]

In [43]:
x_train.shape

(2, 224, 224, 3)

In [44]:
y_train.shape

(2,)

# 建立model

In [244]:
# Instantiate a Keras inception v3 model.
#1. 注意weights 的選用 None or imagenet
#2. include_top 要為False
#3. Assign input shape: 圖片大小
#4. Classes 數量
keras_resnet50 =tf.keras.applications.resnet50.ResNet50(weights=None, include_top=False, pooling=False, 
                                                         input_shape=(224,224,3),classes=15)
#1. 建立最後一層的output
x = keras_resnet50.output
#2. 建立Dense 壓縮資訊量
x = tf.keras.layers.Dense(1024, activation='relu')(x)
#3. 建立預測類別的classes數量
predictions = tf.keras.layers.Dense(15, activation='softmax')(x)
#x = keras_resnet50.output
#x = tf.keras.layers.Flatten()(x)
#predictions = tf.keras.layers.Dense(15, activation='softmax')(x)

In [236]:
# 利用Keras API 來建立learning model
model = tf.keras.Model(inputs=keras_resnet50.input, outputs=predictions)

In [237]:
# Compile model with the optimizer, loss, and metrics you'd like to train with.
# complie 模型:
#1. 指定optimizer
#2. 選用loss function ------ 這點很重要，根據類別(Binary or multi-class)選擇對應的loss function
#3. metric 選擇accuracy
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),
                          loss='sparse_categorical_crossentropy',
                          metric='accuracy')

In [238]:
# Create an Estimator from the compiled Keras model. Note the initial model
# state of the keras model is preserved in the created Estimator.
# 將keras的model 包裝成tensorflow 的estimator
est_keras_resnet50 = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\ADAMCH~1\\AppData\\Local\\Temp\\tmp1fq7ov4c', '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000247CAD519B0>, '_service': None, '_global_id_in_cluster': 0, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_master': '', '_evaluation_master': '', '_save_checkpoints_secs': 600, '_task_id': 0, '_log_step_count_steps': 100, '_session_config': None, '_num_ps_replicas': 0}


In [239]:
# Treat the derived Estimator as you would with any other Estimator.
# First, recover the input name(s) of Keras model, so we can use them as the
# feature column name(s) of the Estimator input function:
# input的key 要用這個名稱來餵入
keras_resnet50.input_names  # print out: ['input_1']

['input_16']

# Train

In [240]:
# Once we have the input name(s), we can create the input function, for example,
# for input(s) in the format of numpy ndarray:

#input_fn 要assign epochs, 
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"input_16": x_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)

In [241]:
# To train, we call Estimator's train function:
# 模型訓練
est_keras_resnet50.train(input_fn=train_input_fn, steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADAMCH~1\AppData\Local\Temp\tmp1fq7ov4c\keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADAMCH~1\AppData\Local\Temp\tmp1fq7ov4c\model.ckpt.
INFO:tensorflow:loss = 9.655557, step = 1
INFO:tensorflow:Loss for final step: 9.655557.


# Evaluation

In [222]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'input_13':x_valid},
    y=y_valid,
    num_epochs=1,
    shuffle=False)

In [223]:
est_keras_resnet50.evaluate(input_fn=eval_input_fn, steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-04-11:18:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADAMCH~1\AppData\Local\Temp\tmp7_t8uta6\model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-06-04-11:18:42
INFO:tensorflow:Saving dict for global step 1: global_step = 1, loss = 2.6740816


{'global_step': 1, 'loss': 2.6740816}

In [224]:
keras_resnet50.output_names

['avg_pool']

# Prediction

In [242]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'input_16':x_test}, num_epochs=1, shuffle=False)

In [243]:
# predict 的iterator 藥用 for 迴圈來call out 執行
pred = est_keras_resnet50.predict(input_fn=test_input_fn)
for r in pred:
    print(r)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADAMCH~1\AppData\Local\Temp\tmp1fq7ov4c\model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'dense_21': array([[[ 0.01528848,  0.01369233, -0.02340276,  0.0344894 ,
         -0.01913951,  0.02997461,  0.00252412,  0.00559476,
          0.03540477,  0.00723353,  0.02681826, -0.00060482,
         -0.00631784, -0.03932065,  0.02564852]]], dtype=float32)}
